In [2]:
# install sdk
!pip install elasticsearch-dsl

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
# 創建客戶端
from elasticsearch import Elasticsearch
es = Elasticsearch(['http://ai-chatbot-es.pri:9200/'])

In [4]:
# 創建index
es.indices.create("test-ai")

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'test-ai'}

In [5]:
# 定義Document長相
from datetime import datetime
from elasticsearch_dsl import Document, Date, Nested, Boolean, analyzer, InnerDoc, Completion, Keyword, Text


class QnA(Document):
    question = Text()
    answer = Text()
    keyword = Text(
        analyzer='standard',
        fields={'raw': Keyword()}
    )
    
    class Index:
        name = 'test-ai'
        
    def save(self, ** kwargs):
        return super().save(** kwargs)

In [6]:
# 創建相應index
QnA.init(using=es)

In [8]:
# 創建Document
qaRecord = QnA(question="今天是好天氣嗎",answer="不是",keyword=["今天","天氣"])

In [9]:
# 給予資料id
qaRecord.meta.id=1234

In [ ]:
# 儲存資料
qaRecord.save(using=es)

In [19]:
# 取得單筆資料
record = QnA.get(using=es, id='1234')
print(record.question)
print(record.keyword)
print(record.answer)

今天是好天氣嗎
['今天', '天氣']
不是


In [27]:
# 比對
qaSearch = QnA.search(using=es)
gg = qaSearch.filter('match',answer="不是")
result = gg.execute()
print(result.to_dict())